# **Modelo de Ciclos de Negócios Reais**

**Introdução aos Modelos DSGE** \\
João Ricardo Costa Filho \\
https://sites.google.com/site/joaoricardocostafilho

## Instale na máquina "virtual" o Octave e o Dynare.

In [ ]:
!apt-get install octave &> /dev/null

In [ ]:
!apt-get install dynare &> /dev/null

## Crie o arquivo .mod

In [ ]:
%%writefile RBC.mod

% Adaptado do livro 'Understanding DSGE Models', Cap. 2.
% note: W e R são variáveis reais na simulação


%--------------------------------------------------------------------------------------------------------------------------------------
% 1. Definição das Variáveis
%--------------------------------------------------------------------------------------------------------------------------------------

var Y I C R K W L A;

varexo e;

%--------------------------------------------------------------------------------------------------------------------------------------
% 2. Calibração
%--------------------------------------------------------------------------------------------------------------------------------------

parameters sigma phi alpha beta delta rhoa;

sigma = 2;
phi   = 1.5;
alpha = 0.35;
beta  = 0.985;
delta = 0.025;
rhoa  = 0.7;

%--------------------------------------------------------------------------------------------------------------------------------------
% 3. Modelo
%--------------------------------------------------------------------------------------------------------------------------------------

model(linear);

#Pbar = 1;
#Rbar = Pbar * ( ( 1 / beta ) - ( 1 - delta ) );
#Wbar = ( 1 - alpha ) * ( Pbar^( 1 / ( 1 - alpha ) ) ) * ( ( alpha / Rbar )^( alpha / ( 1 - alpha ) ) );
#Ybar = ( ( Rbar / (Rbar - delta * alpha ) )^( sigma / ( sigma + phi ) ) ) * ( ( ( 1 - alpha )^( -phi ) )*( ( Wbar / Pbar )^( 1 + phi ) ) )^( 1 / ( sigma + phi ) );
#Kbar = alpha * ( Ybar / Rbar / Pbar );
#Ibar = delta * Kbar;
#Cbar = Ybar - Ibar;
#Lbar = ( 1 - alpha )* ( Ybar / Wbar / Pbar );

%%%%%%%%%%%%% Famílias %%%%%%%%%%%%% 

[name = 'Oferta de Trabalho']
sigma * C + phi * L = W;

[name = 'Equação de Euler']
( sigma / beta ) * ( C(+1) - C ) = Rbar * R(+1);

[name = 'Lei de Movimento do Capital']
K = ( 1 - delta ) * K(-1) + delta * I(-1);

%%%%%%%%%%%%% Empresas %%%%%%%%%%%%% 

[name = 'Função de Produção']
Y = A + alpha*K + ( 1 - alpha ) * L;

[name = 'Demanda por Capital']
R = Y - K;

[name = 'Demanda por Trabalho']
W = Y - L;

%%%%%%%%%%%%% Agragação %%%%%%%%%%%%% 

[name = 'Condição de Equilíbrio']
Ybar * Y = Cbar * C + Ibar * I;

[name = 'Produtividade']
A = rhoa * A(-1) - e;

end;

%--------------------------------------------------------------------------------------------------------------------------------------
% 4. Equilíbrio
%--------------------------------------------------------------------------------------------------------------------------------------

steady;
check;
model_diagnostics;
model_info;

%--------------------------------------------------------------------------------------------------------------------------------------
% 5. Simulação
%--------------------------------------------------------------------------------------------------------------------------------------

shocks;

var e;
stderr 1;
end;

stoch_simul(ar=1, order=1, irf=20);

Writing RBC.mod


In [ ]:
%%writefile RBC.m
dynare RBC.mod


Writing RBC.m


## Resolva e simule o modelo com o Dynare

In [ ]:
!octave RBC.m &> /dev/null

In [ ]:
!dynare RBC.mod &> /dev/null